# Federated Learning for Water Quality Prediction

This notebook demonstrates how to implement federated learning on the water quality dataset using the Flower framework. Federated learning allows training machine learning models across multiple decentralized devices without sharing raw data.

## 1. Install Required Libraries

Install necessary libraries for federated learning implementation.

In [7]:
# Install required libraries if not already installed
%pip install numpy pandas scikit-learn joblib matplotlib seaborn ipykernel keras flwr

  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.3.2-py3-none-any.whl.metadata (5.8 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 M


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

## 2. Prepare Federated Data

Load the dataset and simulate decentralized data across multiple clients (devices).

In [9]:
# Load the dataset
df = pd.read_csv('synthetic_dataset.csv')

# Create target variable: 1 if unsafe (has alerts or abnormal readings), 0 if safe
def create_target(row):
    # Check for explicit alerts (alert column is not NaN)
    if pd.notna(row['alert']):
        return 1
    
    # Check for abnormal sensor statuses
    if row['pressure_status'] in ['High', 'Low']:
        return 1
    if row['tds_status'] == 'Poor':
        return 1
    if row['ph_status'] in ['Acidic', 'Alkaline']:
        return 1
    if row['sensor_status'] == 'Fault':
        return 1
    
    # If none of the above, consider it safe
    return 0

df['unsafe'] = df.apply(create_target, axis=1)

print("Target distribution:")
print(df['unsafe'].value_counts())
print(f"Percentage unsafe: {df['unsafe'].mean()*100:.2f}%")

# Select features
numerical_cols = ['pressure_bar', 'flow_rate_L_min', 'total_volume_L', 'tds_ppm', 'ph', 'temperature_C', 'signal_strength_dBm']
categorical_cols = ['pressure_status', 'tds_status', 'ph_status', 'wifi_status', 'sensor_status']

# Preprocess
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Encode categorical variables
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Features list
features = numerical_cols + [col for col in df_encoded.columns if col.startswith(tuple(categorical_cols)) and col != 'alert']

# Since we only have one device, simulate multiple clients by splitting the data randomly
num_clients = 5
client_data = {}

# Shuffle the data
df_shuffled = df_encoded.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into clients
samples_per_client = len(df_shuffled) // num_clients

for i in range(num_clients):
    start_idx = i * samples_per_client
    end_idx = (i + 1) * samples_per_client if i < num_clients - 1 else len(df_shuffled)
    client_df = df_shuffled.iloc[start_idx:end_idx]
    
    X = client_df[features]
    y = client_df['unsafe']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    client_data[f'client_{i}'] = {
        'X_train': X_train.values.astype(np.float32),
        'X_test': X_test.values.astype(np.float32),
        'y_train': y_train.values.astype(np.float32),
        'y_test': y_test.values.astype(np.float32)
    }

print(f"Simulated {num_clients} federated clients from the dataset")
print("Federated data prepared!")

Target distribution:
unsafe
0    7790
1     850
Name: count, dtype: int64
Percentage unsafe: 9.84%
Simulated 5 federated clients from the dataset
Federated data prepared!


## 3. Define the Global Model

Create a neural network model that will be trained across federated clients.

In [10]:
def create_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(len(features),),
              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(16, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(8, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

# Create global model
global_model = create_model()
global_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,185 (4.63 KB)

 Trainable params: 1,185 (4.63 KB)

 Non-trainable params: 0 (0.00 B)

## 4. Set Up Federated Learning Clients

Define Flower clients that will train locally on their data.

In [11]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, client_id, data):
        self.client_id = client_id
        self.data = data
        self.model = create_model()

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)

        # Local training with early stopping to prevent overfitting
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True,
                                   min_delta=0.001)
        history = self.model.fit(
            self.data['X_train'], self.data['y_train'],
            epochs=20, batch_size=32, validation_split=0.2,
            callbacks=[early_stop], verbose=0
        )

        return self.model.get_weights(), len(self.data['X_train']), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy, precision, recall = self.model.evaluate(self.data['X_test'], self.data['y_test'], verbose=0)
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        return loss, len(self.data['X_test']), {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score
        }

# Create clients
clients = []
for i, (device_id, data) in enumerate(client_data.items()):
    client = FlowerClient(f"client_{i}", data)
    clients.append(client)

print(f"Created {len(clients)} federated clients")

Created 5 federated clients


## 5. Implement Federated Averaging

Set up the federated learning strategy with FedAvg.

In [12]:
def client_fn(cid: str) -> fl.client.Client:
    """Create a Flower client representing a single organization."""
    # Create a Flower client using the client_id
    client_id = int(cid)
    if client_id < len(clients):
        return clients[client_id]
    else:
        raise ValueError(f"Client {client_id} not available")

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1.0,  # Sample 100% of available clients for evaluation
    min_fit_clients=len(clients),  # Never sample fewer than this number of clients for training
    min_evaluate_clients=len(clients),  # Never sample fewer than this number of clients for evaluation
    min_available_clients=len(clients),  # Wait until all clients are available
)

## 6. Train the Model Federally

Run the federated training simulation.

In [13]:
# For demonstration, let's simulate federated learning manually
# In a real setup, you'd run clients and server separately

print("Simulating federated learning rounds...")

# Initialize global model
global_model = create_model()
global_weights = global_model.get_weights()

num_rounds = 5  # More rounds for better convergence

for round_num in range(num_rounds):
    print(f"\n--- Round {round_num + 1} ---")

    # Collect weights from all clients
    client_weights = []
    round_metrics = []

    for client in clients:
        # Simulate local training
        client.model.set_weights(global_weights)

        # Local training
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, min_delta=0.001)
        history = client.model.fit(
            client.data['X_train'], client.data['y_train'],
            epochs=20, batch_size=32, validation_split=0.2,
            callbacks=[early_stop], verbose=0
        )

        client_weights.append(client.model.get_weights())

        # Evaluate locally
        loss, accuracy, precision, recall = client.model.evaluate(client.data['X_test'], client.data['y_test'], verbose=0)
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        round_metrics.append({
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score
        })
        print(f"Client {client.client_id}: Acc={accuracy:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, F1={f1_score:.4f}")

    # Federated averaging (simple average)
    new_weights = []
    for layer_weights in zip(*client_weights):
        # Average weights across clients
        avg_weights = np.mean(layer_weights, axis=0)
        new_weights.append(avg_weights)

    # Update global model
    global_weights = new_weights
    global_model.set_weights(global_weights)

    # Print round summary
    avg_metrics = {k: np.mean([m[k] for m in round_metrics]) for k in round_metrics[0].keys()}
    print(f"Round {round_num + 1} average: Acc={avg_metrics['accuracy']:.4f}, F1={avg_metrics['f1_score']:.4f}")

print("\nFederated learning simulation completed!")

Simulating federated learning rounds...

--- Round 1 ---
Client client_0: Acc=0.9624, Prec=1.0000, Rec=0.6176, F1=0.7636
Client client_1: Acc=0.9480, Prec=1.0000, Rec=0.5000, F1=0.6667
Client client_2: Acc=0.9451, Prec=1.0000, Rec=0.4062, F1=0.5778
Client client_3: Acc=0.9364, Prec=1.0000, Rec=0.3714, F1=0.5417
Client client_4: Acc=0.9451, Prec=1.0000, Rec=0.4242, F1=0.5957
Round 1 average: Acc=0.9474, F1=0.6291

--- Round 2 ---
Client client_0: Acc=0.9913, Prec=1.0000, Rec=0.9118, F1=0.9538
Client client_1: Acc=0.9971, Prec=1.0000, Rec=0.9722, F1=0.9859
Client client_2: Acc=0.9942, Prec=1.0000, Rec=0.9375, F1=0.9677
Client client_3: Acc=0.9913, Prec=1.0000, Rec=0.9143, F1=0.9552
Client client_4: Acc=0.9884, Prec=1.0000, Rec=0.8788, F1=0.9355
Round 2 average: Acc=0.9925, F1=0.9596

--- Round 3 ---
Client client_0: Acc=0.9913, Prec=1.0000, Rec=0.9118, F1=0.9538
Client client_1: Acc=0.9971, Prec=1.0000, Rec=0.9722, F1=0.9859
Client client_2: Acc=0.9942, Prec=1.0000, Rec=0.9375, F1=0.9677

## 7. Evaluate Federated Model Performance

Test the trained global model on a held-out dataset.

In [14]:
# After federated training, the global model weights are updated
# For evaluation, we can use one of the clients or create a test set

# Create a global test set from all devices
all_X_test = np.concatenate([client_data[device]['X_test'] for device in client_data.keys()])
all_y_test = np.concatenate([client_data[device]['y_test'] for device in client_data.keys()])

# Evaluate the global model
loss, accuracy, precision, recall = global_model.evaluate(all_X_test, all_y_test, verbose=0)
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("Global Model Performance:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

# Predictions
y_pred = (global_model.predict(all_X_test) > 0.5).astype(int).flatten()

# Check unique classes
unique_classes = np.unique(all_y_test)
print(f"\nTest set class distribution: {np.bincount(all_y_test.astype(int))}")
print(f"Predicted class distribution: {np.bincount(y_pred)}")

if len(unique_classes) > 1:
    print("\nClassification Report:")
    print(classification_report(all_y_test, y_pred, target_names=['Safe', 'Unsafe']))
else:
    print(f"Only one class present in test data: {unique_classes[0]}")

print("\nFederated learning completed successfully!")
print("The model now predicts water quality as Safe (0) or Unsafe (1)")

Global Model Performance:
Accuracy: 0.9971
Precision: 1.0000
Recall: 0.9706
F1-Score: 0.9851
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Test set class distribution: [1560  170]
Predicted class distribution: [1565  165]

Classification Report:
              precision    recall  f1-score   support

        Safe       1.00      1.00      1.00      1560
      Unsafe       1.00      0.97      0.99       170

    accuracy                           1.00      1730
   macro avg       1.00      0.99      0.99      1730
weighted avg       1.00      1.00      1.00      1730


Federated learning completed successfully!
The model now predicts water quality as Safe (0) or Unsafe (1)


In [15]:
# Save the trained federated model
global_model.save('federated_water_quality_model.h5')
print("Model saved as 'federated_water_quality_model.h5'")

# Save the scaler for future predictions
import joblib
joblib.dump(scaler, 'water_quality_scaler.pkl')
print("Scaler saved as 'water_quality_scaler.pkl'")

# print("\n" + "="*50)
# print("FEDERATED LEARNING SUMMARY")
# print("="*50)
# print("✅ Fixed overfitting issue by correcting target variable logic")
# print("✅ Added L2 regularization and dropout to prevent overfitting")
# print("✅ Implemented proper evaluation metrics (Precision, Recall, F1)")
# print("✅ Balanced dataset: 90.16% Safe, 9.84% Unsafe")
# print("✅ Final model performance:")
# print(f"   Accuracy: {accuracy:.4f}")
# print(f"   Precision: {precision:.4f}")
# print(f"   Recall: {recall:.4f}")
# print(f"   F1-Score: {f1_score:.4f}")
# print("✅ Model predicts: 0 = Safe water, 1 = Unsafe water")
# print("✅ Ready for deployment in federated IoT water monitoring system")

Model saved as 'federated_water_quality_model.h5'
Scaler saved as 'water_quality_scaler.pkl'
